#### Supervisor's Advice
- **Higher Dimensions:** I suggest considering higher dimensions for this implementation. For instance, you could use **time** as a third attribute to evaluate the performance of different indices in accelerating query processing.
- **Query Definition:** Please ensure that you formally define your queries in the Assignment 2 report.|

### Project Flow

**Dataset Preparation** $\rightarrow$ **Algorithm Implementation** $\rightarrow$ **Comparison Experiments**

In [10]:
import pandas as pd
import os

def build_dataset(file_path: str) -> pd.DataFrame:
    """
    Reads a CSV file and constructs a dataset with Latitude and Longitude.
    Returns None if the dataset cannot be built.
    """
    # Check if file exists
    if not os.path.exists(file_path):
        print(f"Error: File not found at {file_path}")
        return None

    # Extract latitude and longitude columns
    # Note: The actual column names in the CSV file are 'decimalLatitude' and 'decimalLongitude' (starting with lowercase)
    required_columns = ['decimalLatitude', 'decimalLongitude']

    # Read CSV file with only specific columns to avoid DtypeWarning and save memory
    try:
        df = pd.read_csv(file_path, usecols=required_columns)
        
        # Construct dataset containing only latitude and longitude, and remove null values
        dataset = df.dropna()
        
        # Rename columns for easier use later (optional)
        dataset.columns = ['Latitude', 'Longitude']
        
        print("Dataset constructed successfully.")
        print(dataset.head())
        print(f"\nShape of dataset: {dataset.shape}")
        return dataset
        
    except ValueError as e:
        print(f"Error reading columns: {e}")
        # Fallback to reading just the header to show available columns
        try:
            df_header = pd.read_csv(file_path, nrows=0)
            print(f"Available columns: {df_header.columns.tolist()}")
        except Exception:
            pass
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None

In [11]:
# Build Platypus dataset
platypus_file_path = 'Dataset/Platypus/Platypus.csv'
platypus_dataset = build_dataset(platypus_file_path)

# Build Legless Lizard dataset
legless_file_path = 'Dataset/Legless_Lizards/Legless_Lizards.csv'
legless_dataset = build_dataset(legless_file_path)

Dataset constructed successfully.
    Latitude   Longitude
0 -34.283333  119.450000
1 -38.155730  144.293950
2 -38.150160  144.301090
3 -38.082467  144.281096
4 -38.147793  144.311988

Shape of dataset: (29656, 2)
Dataset constructed successfully.
    Latitude   Longitude
0 -33.756980  150.628200
1 -33.601780  150.829319
2 -33.740972  150.741930
3 -34.170950  150.612113
4 -33.762350  150.831433

Shape of dataset: (1454283, 2)
Dataset constructed successfully.
    Latitude   Longitude
0 -33.756980  150.628200
1 -33.601780  150.829319
2 -33.740972  150.741930
3 -34.170950  150.612113
4 -33.762350  150.831433

Shape of dataset: (1454283, 2)


In [ ]:
!pip install geopandas fiona

In [17]:
import geopandas as gpd
import fiona

def build_polygon_dataset(gdb_path: str, layer_name: str = None) -> gpd.GeoDataFrame:
    """
    Reads a GDB file and constructs a GeoDataFrame containing polygons.
    If layer_name is not provided, reads the first layer found.
    """
    # Check if the GDB directory exists
    if not os.path.exists(gdb_path):
        print(f"Error: GDB not found at {gdb_path}")
        return None
        
    try:
        # List all layers in the GDB to understand the structure
        layers = fiona.listlayers(gdb_path)
        print(f"Layers found in {os.path.basename(gdb_path)}: {layers}")
        
        if not layers:
            print("Error: No layers found in the GDB.")
            return None
            
        # Determine which layer to read (default to the first one if not specified)
        target_layer = layer_name if layer_name else layers[0]
        
        if target_layer not in layers:
             print(f"Error: Layer '{target_layer}' not found in GDB.")
             return None
             
        print(f"Reading layer: '{target_layer}'...")
        
        # Read the layer into a GeoDataFrame
        gdf = gpd.read_file(gdb_path, layer=target_layer)
        
        print("Dataset constructed successfully.")
        print(f"Shape: {gdf.shape}")
        print(f"Geometry Type: {gdf.geom_type.unique()}")
        print(f"CRS (Coordinate Reference System): {gdf.crs}")
        print(gdf.head())
        return gdf

    except ImportError:
        print("Error: 'geopandas' or 'fiona' library is not installed. Please install them using 'pip install geopandas fiona'.")
        return None
    except Exception as e:
        print(f"An error occurred while reading the GDB: {e}")
        return None

# Define paths to the GDB files
protected_areas_gdb = 'Dataset/Protected_Areas_of_Queensland/data.gdb'
wetlands_gdb = 'Dataset/Wetlands_data_QueenLand/data.gdb'

# Build the datasets
print("--- Building Protected Areas Dataset ---")
protected_areas_gdf = build_polygon_dataset(protected_areas_gdb)

print("\n--- Building Wetlands Dataset ---")
wetlands_gdf = build_polygon_dataset(wetlands_gdb)

--- Building Protected Areas Dataset ---
Layers found in data.gdb: ['Protected_areas']
Reading layer: 'Protected_areas'...
Dataset constructed successfully.
Shape: (2214, 26)
Geometry Type: ['MultiPolygon']
CRS (Coordinate Reference System): EPSG:7844
  lot     plan   lotplans  sysintcode                   estatename  \
0  28    SB653    28SB653  0691ABB001  Abbot Bay Conservation Park   
1   1  AP22467   1AP22467  3336ABE001      Abergowrie State Forest   
2   2  AP22467   2AP22467  3336ABE001      Abergowrie State Forest   
3   5  AP22467   5AP22467  3336ABE001      Abergowrie State Forest   
4  10  AP22467  10AP22467  3336ABE001      Abergowrie State Forest   

      nameabbrev                     namecaps esttype     dcdbtenure  \
0   Abbot Bay CP  ABBOT BAY CONSERVATION PARK      CP  National Park   
1  Abergowrie SF      ABERGOWRIE STATE FOREST      SF   State Forest   
2  Abergowrie SF      ABERGOWRIE STATE FOREST      SF   State Forest   
3  Abergowrie SF      ABERGOWRIE STATE 

### Query Definitions (SQL-like)

**(1) Find the number of sightings of legless lizards in Pine Ridge Conservation Park.**
```sql
SELECT COUNT(*) 
FROM LeglessLizards AS L 
JOIN ProtectedAreas AS P ON ST_Contains(P.geometry, L.geometry) 
WHERE P.estatename = 'Pine Ridge Conservation Park';
```

**(2) Find all wetlands inside a state or national forest park.**
```sql
SELECT W.* 
FROM Wetlands AS W 
JOIN ProtectedAreas AS P ON ST_Within(W.geometry, P.geometry) 
WHERE P.dcdbtenure IN ('State Forest', 'National Park');
```

**(3) Find all sightings of platypus and the distance to the closest wetlands (set the distance to 0 if the sighting is inside a wetland).**
```sql
SELECT P.*, MIN(ST_Distance(P.geometry, W.geometry)) AS distance_to_wetland
FROM Platypus AS P
CROSS JOIN Wetlands AS W
GROUP BY P.id;
```

In [19]:
# --- Data Preprocessing ---

# 1. Convert CSV DataFrames to GeoDataFrames
# Assuming the CSV coordinates are in WGS84 (EPSG:4326)
legless_gdf = gpd.GeoDataFrame(
    legless_dataset, 
    geometry=gpd.points_from_xy(legless_dataset.Longitude, legless_dataset.Latitude),
    crs="EPSG:4326"
)

platypus_gdf = gpd.GeoDataFrame(
    platypus_dataset, 
    geometry=gpd.points_from_xy(platypus_dataset.Longitude, platypus_dataset.Latitude),
    crs="EPSG:4326"
)

# 2. Reproject all datasets to a common projected CRS for accurate spatial operations (e.g., distance)
# EPSG:3577 (GDA94 / Australian Albers) is a good choice for Australia-wide data
target_crs = "EPSG:3577"

print("Reprojecting datasets to EPSG:3577...")
legless_gdf = legless_gdf.to_crs(target_crs)
platypus_gdf = platypus_gdf.to_crs(target_crs)
protected_areas_gdf = protected_areas_gdf.to_crs(target_crs)
wetlands_gdf = wetlands_gdf.to_crs(target_crs)

print("Preprocessing complete.")

Reprojecting datasets to EPSG:3577...
Preprocessing complete.
Preprocessing complete.


In [20]:
# --- Query 1: Number of sightings of legless lizards in Pine Ridge Conservation Park ---

# Filter for the specific park
pine_ridge_park = protected_areas_gdf[protected_areas_gdf['estatename'] == 'Pine Ridge Conservation Park']

if not pine_ridge_park.empty:
    # Perform spatial join: Find points WITHIN the polygon
    # predicate='within' checks if the point is inside the polygon
    lizards_in_park = gpd.sjoin(legless_gdf, pine_ridge_park, how='inner', predicate='within')
    
    count_lizards = len(lizards_in_park)
    print(f"Query 1 Result: Number of legless lizard sightings in Pine Ridge Conservation Park: {count_lizards}")
else:
    print("Query 1 Error: 'Pine Ridge Conservation Park' not found in the protected areas dataset.")

Query 1 Result: Number of legless lizard sightings in Pine Ridge Conservation Park: 32


In [21]:
# --- Query 2: Find all wetlands inside a state or national forest park ---

# Filter for State Forest or National Park
# Note: Checking 'dcdbtenure' column based on previous output inspection
target_parks = protected_areas_gdf[protected_areas_gdf['dcdbtenure'].isin(['State Forest', 'National Park'])]

# Perform spatial join: Find wetlands WITHIN the target parks
# predicate='within' checks if the wetland geometry is completely inside the park geometry
wetlands_in_parks = gpd.sjoin(wetlands_gdf, target_parks, how='inner', predicate='within')

print(f"Query 2 Result: Found {len(wetlands_in_parks)} wetlands inside State or National Forest Parks.")
print(wetlands_in_parks[['wname', 'estatename', 'dcdbtenure']].head())

Query 2 Result: Found 16 wetlands inside State or National Forest Parks.
                    wname                                         estatename  \
66  Hilda Creek Headwater  Daintree National Park (Cape York Peninsula Ab...   
79           Lake Barrine                         Crater Lakes National Park   
81            Lake Eacham                         Crater Lakes National Park   
82         Nandroya Falls                         Wooroonooran National Park   
86  Diamantina Lakes Area                           Diamantina National Park   

       dcdbtenure  
66  National Park  
79  National Park  
81  National Park  
82  National Park  
86  National Park  


In [22]:
# --- Query 3: Find all sightings of platypus and the distance to the closest wetlands ---

# Use sjoin_nearest to find the nearest wetland for each platypus sighting
# distance_col='distance_to_wetland' will store the calculated distance
# If the point is inside a wetland, the distance will be 0
platypus_with_distance = gpd.sjoin_nearest(
    platypus_gdf, 
    wetlands_gdf, 
    how='left', 
    distance_col='distance_to_wetland'
)

print("Query 3 Result: Calculated distance to closest wetland for all platypus sightings.")
# Display relevant columns: Latitude, Longitude (original), and the calculated distance
print(platypus_with_distance[['Latitude', 'Longitude', 'wname', 'distance_to_wetland']].head())

# Optional: Check if any are inside (distance == 0)
inside_count = len(platypus_with_distance[platypus_with_distance['distance_to_wetland'] == 0])
print(f"\nNumber of sightings inside a wetland: {inside_count}")

Query 3 Result: Calculated distance to closest wetland for all platypus sightings.
    Latitude   Longitude                 wname  distance_to_wetland
0 -34.283333  119.450000  Muncoonie Lakes Area         2.083351e+06
1 -38.155730  144.293950           Bulloo Lake         1.032990e+06
2 -38.150160  144.301090           Bulloo Lake         1.032487e+06
3 -38.082467  144.281096           Bulloo Lake         1.024839e+06
4 -38.147793  144.311988           Bulloo Lake         1.032387e+06

Number of sightings inside a wetland: 50


In [23]:
# --- Custom QuadTree Implementation ---

class Rect:
    def __init__(self, xmin, ymin, xmax, ymax):
        self.xmin = xmin
        self.ymin = ymin
        self.xmax = xmax
        self.ymax = ymax
    
    def intersects(self, other):
        return not (other.xmin > self.xmax or other.xmax < self.xmin or
                    other.ymin > self.ymax or other.ymax < self.ymin)
    
    def contains_point(self, x, y):
        return self.xmin <= x <= self.xmax and self.ymin <= y <= self.ymax

    def contains_rect(self, other):
        return (self.xmin <= other.xmin and self.xmax >= other.xmax and
                self.ymin <= other.ymin and self.ymax >= other.ymax)

class QuadTree:
    def __init__(self, boundary, capacity=4, depth=0, max_depth=10):
        self.boundary = boundary  # Rect
        self.capacity = capacity
        self.objects = []  # List of tuples (geometry, data_index/id)
        self.divided = False
        self.depth = depth
        self.max_depth = max_depth
        
        # Children
        self.nw = None
        self.ne = None
        self.sw = None
        self.se = None

    def subdivide(self):
        x_mid = (self.boundary.xmin + self.boundary.xmax) / 2
        y_mid = (self.boundary.ymin + self.boundary.ymax) / 2
        
        self.nw = QuadTree(Rect(self.boundary.xmin, y_mid, x_mid, self.boundary.ymax), self.capacity, self.depth+1, self.max_depth)
        self.ne = QuadTree(Rect(x_mid, y_mid, self.boundary.xmax, self.boundary.ymax), self.capacity, self.depth+1, self.max_depth)
        self.sw = QuadTree(Rect(self.boundary.xmin, self.boundary.ymin, x_mid, y_mid), self.capacity, self.depth+1, self.max_depth)
        self.se = QuadTree(Rect(x_mid, self.boundary.ymin, self.boundary.xmax, y_mid), self.capacity, self.depth+1, self.max_depth)
        self.divided = True

    def insert(self, geometry, data):
        # Get bounding box of the geometry
        minx, miny, maxx, maxy = geometry.bounds
        item_bbox = Rect(minx, miny, maxx, maxy)

        # If not in boundary, ignore
        if not self.boundary.intersects(item_bbox):
            return False

        # MX-CIF Style: If object fits into a child, push it down.
        # Otherwise (overlaps split lines), keep it here.
        
        if self.divided:
            if self.nw.boundary.contains_rect(item_bbox):
                return self.nw.insert(geometry, data)
            elif self.ne.boundary.contains_rect(item_bbox):
                return self.ne.insert(geometry, data)
            elif self.sw.boundary.contains_rect(item_bbox):
                return self.sw.insert(geometry, data)
            elif self.se.boundary.contains_rect(item_bbox):
                return self.se.insert(geometry, data)
            # If it doesn't fit strictly in any child, it stays here (or if it's a point on the edge)
        
        # Add to this node
        self.objects.append((geometry, data))
        
        # Check capacity and split if needed (only if not at max depth)
        if len(self.objects) > self.capacity and self.depth < self.max_depth:
            if not self.divided:
                self.subdivide()
                # Re-distribute existing objects
                # Note: In MX-CIF, we only push down objects that strictly fit.
                # Objects that were already here might now fit into children.
                new_objects = []
                for geo, d in self.objects:
                    b_minx, b_miny, b_maxx, b_maxy = geo.bounds
                    b_rect = Rect(b_minx, b_miny, b_maxx, b_maxy)
                    
                    inserted = False
                    if self.nw.boundary.contains_rect(b_rect):
                        inserted = self.nw.insert(geo, d)
                    elif self.ne.boundary.contains_rect(b_rect):
                        inserted = self.ne.insert(geo, d)
                    elif self.sw.boundary.contains_rect(b_rect):
                        inserted = self.sw.insert(geo, d)
                    elif self.se.boundary.contains_rect(b_rect):
                        inserted = self.se.insert(geo, d)
                    
                    if not inserted:
                        new_objects.append((geo, d))
                
                self.objects = new_objects
                
        return True

    def query(self, range_rect, found_items=None):
        if found_items is None:
            found_items = []
            
        if not self.boundary.intersects(range_rect):
            return found_items

        # Check objects in this node
        for geo, data in self.objects:
            minx, miny, maxx, maxy = geo.bounds
            obj_rect = Rect(minx, miny, maxx, maxy)
            if range_rect.intersects(obj_rect):
                found_items.append((geo, data))

        # Recurse
        if self.divided:
            self.nw.query(range_rect, found_items)
            self.ne.query(range_rect, found_items)
            self.sw.query(range_rect, found_items)
            self.se.query(range_rect, found_items)
            
        return found_items

print("QuadTree class defined.")

QuadTree class defined.


In [24]:
# --- Build QuadTrees for Datasets ---

# Helper to get total bounds for the root node
def get_total_bounds(gdf):
    minx, miny, maxx, maxy = gdf.total_bounds
    # Add a small buffer to avoid edge cases
    return Rect(minx - 1, miny - 1, maxx + 1, maxy + 1)

# 1. Build Lizard Tree (Points)
print("Building QuadTree for Legless Lizards...")
lizard_bounds = get_total_bounds(legless_gdf)
lizard_tree = QuadTree(lizard_bounds, capacity=10)
for idx, row in legless_gdf.iterrows():
    lizard_tree.insert(row.geometry, idx)

# 2. Build Wetland Tree (Polygons)
print("Building QuadTree for Wetlands...")
wetland_bounds = get_total_bounds(wetlands_gdf)
wetland_tree = QuadTree(wetland_bounds, capacity=10)
for idx, row in wetlands_gdf.iterrows():
    wetland_tree.insert(row.geometry, row) # Storing the whole row as data for easy access

print("Trees built successfully.")

Building QuadTree for Legless Lizards...
Building QuadTree for Wetlands...
Trees built successfully.
Building QuadTree for Wetlands...
Trees built successfully.


In [25]:
# --- Execute Queries using QuadTree ---

# Query 1: Legless Lizards in Pine Ridge Conservation Park
print("\n--- Query 1 (QuadTree) ---")
pine_ridge = protected_areas_gdf[protected_areas_gdf['estatename'] == 'Pine Ridge Conservation Park'].iloc[0]
pr_minx, pr_miny, pr_maxx, pr_maxy = pine_ridge.geometry.bounds
search_rect = Rect(pr_minx, pr_miny, pr_maxx, pr_maxy)

# 1. Filter candidates using QuadTree
candidates = lizard_tree.query(search_rect)
print(f"Candidates found by QuadTree: {len(candidates)}")

# 2. Refine using exact geometry check
count_q1 = 0
for geo, idx in candidates:
    if pine_ridge.geometry.contains(geo):
        count_q1 += 1
print(f"Final Count (Legless Lizards in Pine Ridge): {count_q1}")


# Query 2: Wetlands inside State/National Parks
print("\n--- Query 2 (QuadTree) ---")
target_parks = protected_areas_gdf[protected_areas_gdf['dcdbtenure'].isin(['State Forest', 'National Park'])]
wetlands_inside = []

for idx, park in target_parks.iterrows():
    p_minx, p_miny, p_maxx, p_maxy = park.geometry.bounds
    search_rect = Rect(p_minx, p_miny, p_maxx, p_maxy)
    
    # 1. Filter candidates
    candidates = wetland_tree.query(search_rect)
    
    # 2. Refine
    for wet_geo, wet_data in candidates:
        # Avoid duplicates if a wetland is inside multiple parks (unlikely but possible) or checked multiple times
        if park.geometry.contains(wet_geo):
            wetlands_inside.append(wet_data['wname'])

# Remove duplicates
unique_wetlands = set(wetlands_inside)
print(f"Found {len(unique_wetlands)} unique wetlands inside State/National Parks.")


# Query 3: Platypus to Nearest Wetland
print("\n--- Query 3 (QuadTree - Nearest Neighbor) ---")
results_q3 = []

for idx, platypus in platypus_gdf.iterrows():
    p_point = platypus.geometry
    px, py = p_point.x, p_point.y
    
    # Iterative search: Start small, expand if needed
    # Initial radius: 1000m (1km)
    radius = 1000
    found_nearest = False
    min_dist = float('inf')
    nearest_wetland_name = None
    
    # Loop until we find something or radius is huge
    while not found_nearest and radius < 100000: # Max 100km search
        search_rect = Rect(px - radius, py - radius, px + radius, py + radius)
        candidates = wetland_tree.query(search_rect)
        
        if candidates:
            # Check distances
            for wet_geo, wet_data in candidates:
                dist = p_point.distance(wet_geo)
                if dist < min_dist:
                    min_dist = dist
                    nearest_wetland_name = wet_data['wname']
            
            # Verify we didn't miss a closer one just outside the current box
            # If min_dist is within our search radius, we are good.
            # If min_dist > radius (e.g. found in corner of box), we might have missed a closer one in the expanded area.
            # Actually, since we search a square box of size 2*radius, 
            # if min_dist < radius, then the circle of min_dist is fully inside the box. Safe.
            if min_dist < radius:
                found_nearest = True
            else:
                radius *= 2 # Expand and try again to be safe
        else:
            radius *= 2 # Double the search radius
            
    results_q3.append({
        'Platypus_ID': idx,
        'Nearest_Wetland': nearest_wetland_name,
        'Distance': min_dist if min_dist != float('inf') else None
    })

# Convert to DataFrame for display
df_q3 = pd.DataFrame(results_q3)
print(df_q3.head())
print(f"Processed {len(df_q3)} platypus sightings.")


--- Query 1 (QuadTree) ---
Candidates found by QuadTree: 22
Final Count (Legless Lizards in Pine Ridge): 20

--- Query 2 (QuadTree) ---
Found 11 unique wetlands inside State/National Parks.

--- Query 3 (QuadTree - Nearest Neighbor) ---
   Platypus_ID Nearest_Wetland  Distance
0            0            None       NaN
1            1            None       NaN
2            2            None       NaN
3            3            None       NaN
4            4            None       NaN
Processed 29656 platypus sightings.
   Platypus_ID Nearest_Wetland  Distance
0            0            None       NaN
1            1            None       NaN
2            2            None       NaN
3            3            None       NaN
4            4            None       NaN
Processed 29656 platypus sightings.


In [26]:
# --- Custom R-Tree Implementation ---

class RTreeNode:
    def __init__(self, is_leaf=True, capacity=4):
        self.is_leaf = is_leaf
        self.capacity = capacity
        self.children = []  # List of (bbox, child_node_or_data)
        self.bbox = None    # Bounding box of this node

    def update_bbox(self):
        if not self.children:
            self.bbox = None
            return

        minx = min(child[0][0] for child in self.children)
        miny = min(child[0][1] for child in self.children)
        maxx = max(child[0][2] for child in self.children)
        maxy = max(child[0][3] for child in self.children)
        self.bbox = (minx, miny, maxx, maxy)

    def insert(self, item_bbox, item_data):
        if self.is_leaf:
            self.children.append((item_bbox, item_data))
            self.update_bbox()
            if len(self.children) > self.capacity:
                return self.split()
            return None
        else:
            # Choose best child to insert into (least enlargement)
            best_child_idx = self._choose_subtree(item_bbox)
            new_child = self.children[best_child_idx][1].insert(item_bbox, item_data)
            
            if new_child:
                self.children.append(new_child)
                if len(self.children) > self.capacity:
                    split_result = self.split()
                    self.update_bbox() # Update after split
                    return split_result
            
            # Update bbox of the chosen child in the children list
            child_node = self.children[best_child_idx][1]
            self.children[best_child_idx] = (child_node.bbox, child_node)
            self.update_bbox()
            return None

    def _choose_subtree(self, item_bbox):
        best_idx = -1
        min_enlargement = float('inf')
        
        for i, (child_bbox, _) in enumerate(self.children):
            # Calculate enlargement
            enlarged_bbox = (
                min(child_bbox[0], item_bbox[0]),
                min(child_bbox[1], item_bbox[1]),
                max(child_bbox[2], item_bbox[2]),
                max(child_bbox[3], item_bbox[3])
            )
            current_area = (child_bbox[2] - child_bbox[0]) * (child_bbox[3] - child_bbox[1])
            new_area = (enlarged_bbox[2] - enlarged_bbox[0]) * (enlarged_bbox[3] - enlarged_bbox[1])
            enlargement = new_area - current_area
            
            if enlargement < min_enlargement:
                min_enlargement = enlargement
                best_idx = i
        return best_idx

    def split(self):
        # Linear Split Strategy (Simplified)
        # Sort by x-min and split in half
        self.children.sort(key=lambda x: x[0][0])
        mid = len(self.children) // 2
        
        new_node = RTreeNode(is_leaf=self.is_leaf, capacity=self.capacity)
        new_node.children = self.children[mid:]
        self.children = self.children[:mid]
        
        self.update_bbox()
        new_node.update_bbox()
        
        return (new_node.bbox, new_node)

    def query(self, search_bbox, results=None):
        if results is None:
            results = []
            
        if not self.bbox or not self._intersects(self.bbox, search_bbox):
            return results

        if self.is_leaf:
            for item_bbox, item_data in self.children:
                if self._intersects(item_bbox, search_bbox):
                    results.append((item_bbox, item_data))
        else:
            for child_bbox, child_node in self.children:
                if self._intersects(child_bbox, search_bbox):
                    child_node.query(search_bbox, results)
        return results

    def _intersects(self, box1, box2):
        return not (box1[2] < box2[0] or box1[0] > box2[2] or
                    box1[3] < box2[1] or box1[1] > box2[3])

class RTree:
    def __init__(self, capacity=4):
        self.root = RTreeNode(is_leaf=True, capacity=capacity)
        self.capacity = capacity

    def insert(self, geometry, data):
        minx, miny, maxx, maxy = geometry.bounds
        item_bbox = (minx, miny, maxx, maxy)
        
        new_child = self.root.insert(item_bbox, data)
        if new_child:
            # Root split, create new root
            new_root = RTreeNode(is_leaf=False, capacity=self.capacity)
            new_root.children.append((self.root.bbox, self.root))
            new_root.children.append(new_child)
            new_root.update_bbox()
            self.root = new_root

    def query(self, search_bbox):
        return self.root.query(search_bbox)

print("RTree class defined.")

RTree class defined.


In [27]:
# --- Build R-Trees for Datasets ---

# 1. Build Lizard R-Tree
print("Building R-Tree for Legless Lizards...")
lizard_rtree = RTree(capacity=10)
for idx, row in legless_gdf.iterrows():
    lizard_rtree.insert(row.geometry, idx)

# 2. Build Wetland R-Tree
print("Building R-Tree for Wetlands...")
wetland_rtree = RTree(capacity=10)
for idx, row in wetlands_gdf.iterrows():
    wetland_rtree.insert(row.geometry, row)

print("R-Trees built successfully.")

Building R-Tree for Legless Lizards...
Building R-Tree for Wetlands...
R-Trees built successfully.
Building R-Tree for Wetlands...
R-Trees built successfully.


In [28]:
# --- Execute Queries using R-Tree ---

# Query 1: Legless Lizards in Pine Ridge Conservation Park
print("\n--- Query 1 (R-Tree) ---")
pine_ridge = protected_areas_gdf[protected_areas_gdf['estatename'] == 'Pine Ridge Conservation Park'].iloc[0]
pr_minx, pr_miny, pr_maxx, pr_maxy = pine_ridge.geometry.bounds
search_bbox = (pr_minx, pr_miny, pr_maxx, pr_maxy)

# 1. Filter candidates using R-Tree
candidates = lizard_rtree.query(search_bbox)
print(f"Candidates found by R-Tree: {len(candidates)}")

# 2. Refine using exact geometry check
count_q1_rtree = 0
for bbox, idx in candidates:
    # Retrieve geometry from original GDF using index
    geo = legless_gdf.loc[idx].geometry
    if pine_ridge.geometry.contains(geo):
        count_q1_rtree += 1
print(f"Final Count (Legless Lizards in Pine Ridge): {count_q1_rtree}")


# Query 2: Wetlands inside State/National Parks
print("\n--- Query 2 (R-Tree) ---")
target_parks = protected_areas_gdf[protected_areas_gdf['dcdbtenure'].isin(['State Forest', 'National Park'])]
wetlands_inside_rtree = []

for idx, park in target_parks.iterrows():
    p_minx, p_miny, p_maxx, p_maxy = park.geometry.bounds
    search_bbox = (p_minx, p_miny, p_maxx, p_maxy)
    
    # 1. Filter candidates
    candidates = wetland_rtree.query(search_bbox)
    
    # 2. Refine
    for wet_bbox, wet_data in candidates:
        wet_geo = wet_data.geometry
        if park.geometry.contains(wet_geo):
            wetlands_inside_rtree.append(wet_data['wname'])

# Remove duplicates
unique_wetlands_rtree = set(wetlands_inside_rtree)
print(f"Found {len(unique_wetlands_rtree)} unique wetlands inside State/National Parks.")


# Query 3: Platypus to Nearest Wetland
print("\n--- Query 3 (R-Tree - Nearest Neighbor) ---")
results_q3_rtree = []

for idx, platypus in platypus_gdf.iterrows():
    p_point = platypus.geometry
    px, py = p_point.x, p_point.y
    
    # Iterative search: Start small, expand if needed
    radius = 1000
    found_nearest = False
    min_dist = float('inf')
    nearest_wetland_name = None
    
    while not found_nearest and radius < 100000:
        search_bbox = (px - radius, py - radius, px + radius, py + radius)
        candidates = wetland_rtree.query(search_bbox)
        
        if candidates:
            for wet_bbox, wet_data in candidates:
                wet_geo = wet_data.geometry
                dist = p_point.distance(wet_geo)
                if dist < min_dist:
                    min_dist = dist
                    nearest_wetland_name = wet_data['wname']
            
            if min_dist < radius:
                found_nearest = True
            else:
                radius *= 2
        else:
            radius *= 2
            
    results_q3_rtree.append({
        'Platypus_ID': idx,
        'Nearest_Wetland': nearest_wetland_name,
        'Distance': min_dist if min_dist != float('inf') else None
    })

# Convert to DataFrame for display
df_q3_rtree = pd.DataFrame(results_q3_rtree)
print(df_q3_rtree.head())
print(f"Processed {len(df_q3_rtree)} platypus sightings.")


--- Query 1 (R-Tree) ---
Candidates found by R-Tree: 22
Final Count (Legless Lizards in Pine Ridge): 20

--- Query 2 (R-Tree) ---
Found 11 unique wetlands inside State/National Parks.

--- Query 3 (R-Tree - Nearest Neighbor) ---
   Platypus_ID Nearest_Wetland  Distance
0            0            None       NaN
1            1            None       NaN
2            2            None       NaN
3            3            None       NaN
4            4            None       NaN
Processed 29656 platypus sightings.
   Platypus_ID Nearest_Wetland  Distance
0            0            None       NaN
1            1            None       NaN
2            2            None       NaN
3            3            None       NaN
4            4            None       NaN
Processed 29656 platypus sightings.
